In [88]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras import layers, models
import keras

In [89]:
# Define path to the dataset
PATH = "C:/Users/emanu/Universidade/Mestrado/1_Ano/2_Semestre/ACA/Projeto/ACA-Project/AML-Project2/dataset"

# Read the CSV file containing the image paths and labels
# Read the training set data
train_data = np.load(os.path.join(PATH, "trainX.npy"))
train_labels = np.load(os.path.join(PATH, "trainy.npy"), allow_pickle=True)

test_data = np.load(os.path.join(PATH, "testX.npy"))

print(train_data.shape)
display(train_data)

(12000, 50, 50, 3)


array([[[[ 47,  64,  77],
         [ 43,  57,  80],
         [182, 206, 224],
         ...,
         [ 10,   8,   8],
         [  9,   7,   7],
         [  7,   5,   5]],

        [[ 42,  59,  80],
         [ 14,  22,  35],
         [ 50,  62,  74],
         ...,
         [ 14,   9,  10],
         [ 10,   5,   6],
         [ 11,   6,   7]],

        [[ 51,  68,  87],
         [  0,   5,   9],
         [  7,  14,  23],
         ...,
         [ 13,   8,  10],
         [ 13,   8,   9],
         [ 11,   6,   7]],

        ...,

        [[ 57,  63,  76],
         [ 43,  55,  65],
         [ 33,  45,  63],
         ...,
         [ 37,  36,  40],
         [ 17,  12,  14],
         [ 16,  11,  13]],

        [[ 48,  58,  65],
         [ 47,  53,  66],
         [ 45,  53,  60],
         ...,
         [ 19,  21,  22],
         [ 25,  20,  22],
         [ 14,   9,  10]],

        [[ 32,  40,  47],
         [ 13,  19,  30],
         [ 11,  17,  24],
         ...,
         [237, 239, 247],
        

In [90]:
train_data = train_data / 255.0

In [91]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, train_size=0.8, random_state=42, stratify=train_labels)

In [92]:
display(X_train.shape)

(9600, 50, 50, 3)

In [93]:
model = models.Sequential()

model.add(layers.Conv2D(32, 3, activation='relu', input_shape=(50, 50, 3)))
model.add(layers.Conv2D(64, 3, activation='relu'))
model.add(layers.Conv2D(128, 3, activation='relu'))
model.add(layers.MaxPool2D())
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(6, activation='softmax'))

model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_27 (Conv2D)          (None, 48, 48, 32)        896       
                                                                 
 conv2d_28 (Conv2D)          (None, 46, 46, 64)        18496     
                                                                 
 conv2d_29 (Conv2D)          (None, 44, 44, 128)       73856     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 22, 22, 128)      0         
 2D)                                                             
                                                                 
 dropout_9 (Dropout)         (None, 22, 22, 128)       0         
                                                                 
 flatten_9 (Flatten)         (None, 61952)             0         
                                                      

In [94]:
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)

In [95]:
train_Y_one_hot = keras.utils.to_categorical(y_train)

In [96]:
model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    x=X_train,
    y=train_Y_one_hot,
    batch_size=32,
    epochs=20,
    validation_split=0.25
)

Epoch 1/20
225/225 [==============================] - 40s 172ms/step - loss: 1.1311 - accuracy: 0.5374 - val_loss: 0.9089 - val_accuracy: 0.6546
Epoch 2/20
225/225 [==============================] - 42s 187ms/step - loss: 0.8458 - accuracy: 0.6711 - val_loss: 0.7766 - val_accuracy: 0.7279
Epoch 3/20
225/225 [==============================] - 43s 193ms/step - loss: 0.6681 - accuracy: 0.7504 - val_loss: 0.7550 - val_accuracy: 0.7400
Epoch 4/20
225/225 [==============================] - 44s 197ms/step - loss: 0.5147 - accuracy: 0.8143 - val_loss: 0.7134 - val_accuracy: 0.7458
Epoch 5/20
225/225 [==============================] - 45s 200ms/step - loss: 0.4037 - accuracy: 0.8594 - val_loss: 0.7888 - val_accuracy: 0.7429
Epoch 6/20
225/225 [==============================] - 46s 203ms/step - loss: 0.2850 - accuracy: 0.9018 - val_loss: 0.9844 - val_accuracy: 0.7271
Epoch 7/20
225/225 [==============================] - 46s 204ms/step - loss: 0.2046 - accuracy: 0.9272 - val_loss: 1.0352 - val_ac

In [97]:
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
test_Y_one_hot = tf.convert_to_tensor(test_Y_one_hot, dtype=tf.float32)
test_Y_one_hot = keras.utils.to_categorical(y_test)

from sklearn.metrics import classification_report

model.evaluate(X_test, test_Y_one_hot)

pred = model.predict(X_test)
pred = np.argmax(pred, axis=1)

test_Y_one_hot = np.argmax(test_Y_one_hot, axis=1)

print(pred)
print(test_Y_one_hot)

print(classification_report(test_Y_one_hot, pred, zero_division=0))
print('=========================================')

75/75 [==============================] - 3s 40ms/step
[1 0 3 ... 1 1 0]
[1 0 3 ... 1 1 0]
              precision    recall  f1-score   support

           0       0.75      0.68      0.72       400
           1       0.92      0.92      0.92       400
           2       0.79      0.75      0.77       400
           3       0.70      0.76      0.73       400
           4       0.78      0.75      0.76       400
           5       0.72      0.80      0.76       400

    accuracy                           0.78      2400
   macro avg       0.78      0.78      0.77      2400
weighted avg       0.78      0.78      0.77      2400



In [98]:
test_df = pd.read_csv(os.path.join(PATH, "test_intel-image-classification-csvdata-kaggle.csv"))

test_data = test_data / 255.0

test_data = tf.convert_to_tensor(test_data, dtype=tf.float32)

In [99]:
preds = model.predict(test_data)
predict_test = np.argmax(preds, axis=1)

submission = pd.DataFrame({'number': test_df['number'], 'class': predict_test})
print(submission)
submission.to_csv('submission_cnn_oneHot_025val.csv', index=False)

158/158 [==============================] - 6s 39ms/step
      number  class
0          0      1
1          1      0
2          2      0
3          3      5
4          4      0
...      ...    ...
5046    5046      0
5047    5047      0
5048    5048      5
5049    5049      5
5050    5050      5

[5051 rows x 2 columns]
